In [28]:
%pylab inline
import matplotlib.pyplot as plt


Populating the interactive namespace from numpy and matplotlib


/home/navid/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [29]:
import os
from scipy.stats import ttest_rel
import copy
import pickle
import pandas as pd
import copy

from sklearn.preprocessing import MinMaxScaler


import numpy as np
from numpy import linalg as LA
import scipy
from scipy.sparse import csr_matrix
from sklearn import preprocessing

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel

from scipy.stats.stats import spearmanr
from scipy.stats.stats import pearsonr


from sklearn.decomposition import PCA

In [30]:
we_model_name = "sg_dim300_min200_win5"
we_vector_size = 300
we_model_dir = "/share/home/navid/wordembeddings/word2vecgensim/wikipedia/wiki-english-20171001/%s" % we_model_name


In [31]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')
print ('loading done!')

loading done!


In [32]:
wordpairs_debiasing_direction = []
with open('resources/wordpairs_direction.txt') as fr:
    for l in fr:
        wordpairs_debiasing_direction.append(l.strip().split(','))
print (len(wordpairs_debiasing_direction))




28


In [33]:
job_tuples=[]
with open('resources/wordlist_occupations.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        if _tuple[0] in we_model.wv.vocab:
            job_tuples.append(_tuple)
print (len(job_tuples))




497


In [34]:
def Cosine(vec1, vec2):
    return 1 - scipy.spatial.distance.cosine(vec1, vec2)

def meanCosine(vec, set_vecs):
    return np.mean([Cosine(vec, _set_vec) for _set_vec in set_vecs])

def NegDifNorm(vec1, vec2):
    return -np.linalg.norm((vec1/np.linalg.norm(vec1)) - (vec2/np.linalg.norm(vec2)))


In [35]:
coomtx_dir = '/share/home/navid/wordembeddings/cooccurrencematrix/wikipedia/wiki-english-20171001/min200_win5'
coomtx_dict = Dictionary.load(coomtx_dir + '/dictionary.gensim')
_loader = np.load(coomtx_dir + '/cooccurrence_matrix.npz')
coomtx = csr_matrix((_loader['data'], _loader['indices'], _loader['indptr']), shape = _loader['shape'])
with open(coomtx_dir + '/vocabs_count.pkl', 'rb') as fr:
    coomtx_vocabcount = pickle.load(fr)

print ('loading done!')

loading done!


In [36]:
cds=0.75

coomtx_colsum_vec = np.array(coomtx.sum(axis=0), dtype=np.float64)[0]
coomtx_colsum_vec[coomtx_colsum_vec==0] = 1.0
coomtx_colsum_cds_vec = coomtx_colsum_vec ** cds
coomtx_colsum_cds_vec_sum = np.sum(coomtx_colsum_cds_vec)

coomtx_D_cnt=float(coomtx.sum())
print ('done!')

done!


In [37]:
with open(coomtx_dir + '/pmi_svd_matrix.pkl', 'rb') as fr:
    pmisvd_matrix = pickle.load(fr)
with open(coomtx_dir + '/ppmi_svd_matrix.pkl', 'rb') as fr:
    ppmisvd_matrix = pickle.load(fr)
with open(coomtx_dir + '/sppmi_svd_matrix.pkl', 'rb') as fr:
    sppmisvd_matrix = pickle.load(fr)


print ('loading SVDs done!')

loading SVDs done!


In [38]:

def get_cooccurrence_vec(word):
    _idx = coomtx_dict.token2id[word]
    _vec = np.array(coomtx.getrow(_idx).toarray()[0], dtype=np.float64)
    return _vec

def get_cooccurprob_vec(word):
    _coo_vec = get_cooccurrence_vec(word)
    _vec = _coo_vec / np.sum(_coo_vec)
    return _vec


def get_pmi_vec(word):
    _coo_vec = get_cooccurrence_vec(word)
    _vec = np.log((_coo_vec * coomtx_D_cnt) / (np.sum(_coo_vec) * coomtx_colsum_vec))
    return _vec

def get_pmicds_vec(word):
    _coo_vec = get_cooccurrence_vec(word)
    _vec = np.log((_coo_vec) / (np.sum(_coo_vec) * (coomtx_colsum_cds_vec / coomtx_colsum_cds_vec_sum)))
    return _vec

def get_pmibased_vecs(words):
    ns=5.
    
    get_cooccurprob_vec
    
    CoProb_vecs = {}
    PMI_vecs = {}
    PPMI_vecs = {}
    SPPMI_vecs = {}
    for word in words:
        _vec = get_cooccurprob_vec(word)
        _vec[_vec == -np.inf]=0.0
        CoProb_vecs[word] = _vec
        
        _vec = get_pmicds_vec(word)
    
        #PMI
        _pmi_vec = copy.copy(_vec)    
        _pmi_vec[_pmi_vec == -np.inf]=0.0
        PMI_vecs[word] = _pmi_vec
        
        #PPMI
        _ppmi_vec = copy.copy(_vec)    
        _ppmi_vec[_ppmi_vec < 0] = 0
        PPMI_vecs[word] = _ppmi_vec
    
        #SPPMI
        _sppmi_vec = copy.copy(_vec)    
        _sppmi_vec = _sppmi_vec - np.log(ns)
        _sppmi_vec[_sppmi_vec<0] = 0.0
        SPPMI_vecs[word] = _sppmi_vec

    return CoProb_vecs, PMI_vecs, PPMI_vecs, SPPMI_vecs

get_pmibased_vecs(['book'])


/home/navid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


({'book': array([1.89892886e-02, 3.85159410e-06, 7.96881537e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00])},
 {'book': array([ 1.35301275, -1.11489032, -0.26904235, ...,  0.        ,
          0.        ,  0.        ])},
 {'book': array([1.35301275, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ])},
 {'book': array([0., 0., 0., ..., 0., 0., 0.])})

In [39]:

def get_pmisvdbased_vecs(words):
    PMISVD_vecs = {}
    PPMISVD_vecs = {}
    SPPMISVD_vecs = {}
    for word in words:
        _idx = coomtx_dict.token2id[word]
        PMISVD_vecs[word] = pmisvd_matrix[_idx]
        PPMISVD_vecs[word] = ppmisvd_matrix[_idx]
        SPPMISVD_vecs[word] = sppmisvd_matrix[_idx]

    return PMISVD_vecs, PPMISVD_vecs, SPPMISVD_vecs

get_pmisvdbased_vecs(['she'])[0]['she'][:10]

array([ 1.38065485e+01, -3.07478468e+00, -7.86560477e+00,  3.27757522e-01,
       -1.25351224e-01,  5.00331066e-01, -3.28505429e+00, -7.77388278e-01,
       -8.87781231e-03, -4.31638237e+00])

In [40]:
def get_pmibased_vecs_similarity(word_from, words_to):
    CoProb_vecs_from, PMI_vecs_from, PPMI_vecs_from, SPPMI_vecs_from = get_pmibased_vecs([word_from])
    CoProb_vecs_to, PMI_vecs_to, PPMI_vecs_to, SPPMI_vecs_to = get_pmibased_vecs(words_to)
    
    sims_CoProb = {}
    sims_PMI = {}
    sims_PPMI = {}
    sims_SPPMI = {}
    for word_to in words_to:
        if word_to in coomtx_dict.values():
            sims_CoProb[word_to] = Cosine(CoProb_vecs_from[word_from], CoProb_vecs_to[word_to])
            sims_PMI[word_to] = Cosine(PMI_vecs_from[word_from], PMI_vecs_to[word_to])
            sims_PPMI[word_to] = Cosine(PPMI_vecs_from[word_from], PPMI_vecs_to[word_to])
            sims_SPPMI[word_to] = Cosine(SPPMI_vecs_from[word_from], SPPMI_vecs_to[word_to])
            
    return sims_CoProb, sims_PMI, sims_PPMI, sims_SPPMI

get_pmibased_vecs_similarity('book', ['book', 'library'])

/home/navid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


({'book': 1.0, 'library': 0.9578772703233035},
 {'book': 1.0, 'library': 0.43006654138569744},
 {'book': 1.0, 'library': 0.11733015475554986},
 {'book': 1.0, 'library': 0.04354094496879557})

In [41]:
genderword_tuples = []

with open('resources/wordlist_genderspecific.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        if _tuple[0] in we_model.wv.vocab:
            _idx = we_model.wv.vocab[_tuple[0]].index
            genderword_tuples.append((_tuple[0], _tuple[1], _idx))
        else:
            print (_tuple[0])
print (len([x for x in genderword_tuples if x[1]=='f']))
print (len([x for x in genderword_tuples if x[1]=='m']))




28
28


In [42]:
coomtx_job_tuples=[]
with open('resources/wordlist_occupations.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        if _tuple[0] in coomtx_dict.values():
            coomtx_job_tuples.append(_tuple)
        

coomtx_genderword_tuples = []

with open('resources/wordlist_genderspecific.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        if _tuple[0] in coomtx_dict.values():
            _idx = coomtx_dict.token2id[_tuple[0]]
            coomtx_genderword_tuples.append((_tuple[0], _tuple[1], _idx))



In [43]:
coomtx_fem_exp_feature_vec = np.zeros(len(we_model.wv.vocab))
coomtx_masc_exp_feature_vec = np.zeros(len(we_model.wv.vocab))
for _tuple in coomtx_genderword_tuples:
    if _tuple[1] == 'f':
        coomtx_fem_exp_feature_vec[_tuple[2]] = 1
    elif _tuple[1] == 'm':
        coomtx_masc_exp_feature_vec[_tuple[2]] = 1


In [44]:
job_CoProb_vecs, job_PMI_vecs, job_PPMI_vecs, job_SPPMI_vecs = get_pmibased_vecs([x[0] for x in coomtx_job_tuples])

job_PMISVD_vecs, job_PPMISVD_vecs, job_SPPMISVD_vecs = get_pmisvdbased_vecs([x[0] for x in coomtx_job_tuples])



/home/navid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


In [63]:
_CoProb_fem_vectors = []
_CoProb_masc_vectors = []
_PMI_fem_vectors = []
_PMI_masc_vectors = []
_PPMI_fem_vectors = []
_PPMI_masc_vectors = []
_SPPMI_fem_vectors = []
_SPPMI_masc_vectors = []
for _tuple in genderword_tuples:
    _CoProb_vecs, _PMI_vecs, _PPMI_vecs, _SPPMI_vecs = get_pmibased_vecs([_tuple[0]])
    if _tuple[1] == 'f':
        _CoProb_fem_vectors.append(_CoProb_vecs[_tuple[0]])
        _PMI_fem_vectors.append(_PMI_vecs[_tuple[0]])
        _PPMI_fem_vectors.append(_PPMI_vecs[_tuple[0]])
        _SPPMI_fem_vectors.append(_SPPMI_vecs[_tuple[0]])
    elif _tuple[1] == 'm':
        _CoProb_masc_vectors.append(_CoProb_vecs[_tuple[0]])
        _PMI_masc_vectors.append(_PMI_vecs[_tuple[0]])
        _PPMI_masc_vectors.append(_PPMI_vecs[_tuple[0]])
        _SPPMI_masc_vectors.append(_SPPMI_vecs[_tuple[0]])
    
fem_CoProb_centroid_vec = np.mean(np.array(_CoProb_fem_vectors), axis=0)
masc_CoProb_centroid_vec = np.mean(np.array(_CoProb_masc_vectors), axis=0)
fem_PMI_centroid_vec = np.mean(np.array(_PMI_fem_vectors), axis=0)
masc_PMI_centroid_vec = np.mean(np.array(_PMI_masc_vectors), axis=0)
fem_PPMI_centroid_vec = np.mean(np.array(_PPMI_fem_vectors), axis=0)
masc_PPMI_centroid_vec = np.mean(np.array(_PPMI_masc_vectors), axis=0)
fem_SPPMI_centroid_vec = np.mean(np.array(_SPPMI_fem_vectors), axis=0)
masc_SPPMI_centroid_vec = np.mean(np.array(_SPPMI_masc_vectors), axis=0)



/home/navid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


In [46]:
_PMISVD_fem_vectors = []
_PMISVD_masc_vectors = []
_PPMISVD_fem_vectors = []
_PPMISVD_masc_vectors = []
_SPPMISVD_fem_vectors = []
_SPPMISVD_masc_vectors = []
for _tuple in genderword_tuples:
    _PMISVD_vecs, _PPMISVD_vecs, _SPPMISVD_vecs = get_pmisvdbased_vecs([_tuple[0]])
    if _tuple[1] == 'f':
        _PMISVD_fem_vectors.append(_PMISVD_vecs[_tuple[0]])
        _PPMISVD_fem_vectors.append(_PPMISVD_vecs[_tuple[0]])
        _SPPMISVD_fem_vectors.append(_SPPMISVD_vecs[_tuple[0]])
    elif _tuple[1] == 'm':
        _PMISVD_masc_vectors.append(_PMISVD_vecs[_tuple[0]])
        _PPMISVD_masc_vectors.append(_PPMISVD_vecs[_tuple[0]])
        _SPPMISVD_masc_vectors.append(_SPPMISVD_vecs[_tuple[0]])
    
fem_PMISVD_centroid_vec = np.mean(np.array(_PMISVD_fem_vectors), axis=0)
masc_PMISVD_centroid_vec = np.mean(np.array(_PMISVD_masc_vectors), axis=0)
fem_PPMISVD_centroid_vec = np.mean(np.array(_PPMISVD_fem_vectors), axis=0)
masc_PPMISVD_centroid_vec = np.mean(np.array(_PPMISVD_masc_vectors), axis=0)
fem_SPPMISVD_centroid_vec = np.mean(np.array(_SPPMISVD_fem_vectors), axis=0)
masc_SPPMISVD_centroid_vec = np.mean(np.array(_SPPMISVD_masc_vectors), axis=0)


In [59]:
#Gender dirction dense

_matrix_CoProb = []
_matrix_PMI = []
_matrix_PPMI = []
_matrix_SPPMI = []
_matrix_PMISVD = []
_matrix_PPMISVD = []
_matrix_SPPMISVD = []
for a, b in wordpairs_debiasing_direction:
    _CoProb_vecs, _PMI_vecs, _PPMI_vecs, _SPPMI_vecs = get_pmibased_vecs([a, b])
    _a_CoProb_vec_norm = _CoProb_vecs[a]/np.linalg.norm(_CoProb_vecs[a])
    _a_PMI_vec_norm = _PMI_vecs[a]/np.linalg.norm(_PMI_vecs[a])
    _a_PPMI_vec_norm = _PPMI_vecs[a]/np.linalg.norm(_PPMI_vecs[a])
    _a_SPPMI_vec_norm = _SPPMI_vecs[a]/np.linalg.norm(_SPPMI_vecs[a])
    _b_CoProb_vec_norm = _CoProb_vecs[b]/np.linalg.norm(_CoProb_vecs[b])
    _b_PMI_vec_norm = _PMI_vecs[b]/np.linalg.norm(_PMI_vecs[b])
    _b_PPMI_vec_norm = _PPMI_vecs[b]/np.linalg.norm(_PPMI_vecs[b])
    _b_SPPMI_vec_norm = _SPPMI_vecs[b]/np.linalg.norm(_SPPMI_vecs[b])
    
    _center_CoProb_vec = (_a_CoProb_vec_norm + _b_CoProb_vec_norm)/2
    _center_PMI_vec = (_a_PMI_vec_norm + _b_PMI_vec_norm)/2
    _center_PPMI_vec = (_a_PPMI_vec_norm + _b_PPMI_vec_norm)/2
    _center_SPPMI_vec = (_a_SPPMI_vec_norm + _b_SPPMI_vec_norm)/2
    
    _matrix_CoProb.append(_a_CoProb_vec_norm - _center_CoProb_vec)
    _matrix_PMI.append(_a_PMI_vec_norm - _center_PMI_vec)
    _matrix_PPMI.append(_a_PPMI_vec_norm - _center_PPMI_vec)
    _matrix_SPPMI.append(_a_SPPMI_vec_norm - _center_SPPMI_vec)
    _matrix_CoProb.append(_b_CoProb_vec_norm - _center_CoProb_vec)
    _matrix_PMI.append(_b_PMI_vec_norm - _center_PMI_vec)
    _matrix_PPMI.append(_b_PPMI_vec_norm - _center_PPMI_vec)
    _matrix_SPPMI.append(_b_SPPMI_vec_norm - _center_SPPMI_vec)
    
    _PMISVD_vecs, _PPMISVD_vecs, _SPPMISVD_vecs = get_pmisvdbased_vecs([a, b])
    _a_PMISVD_vec_norm = _PMISVD_vecs[a]/np.linalg.norm(_PMISVD_vecs[a])
    _a_PPMISVD_vec_norm = _PPMISVD_vecs[a]/np.linalg.norm(_PPMISVD_vecs[a])
    _a_SPPMISVD_vec_norm = _SPPMISVD_vecs[a]/np.linalg.norm(_SPPMISVD_vecs[a])
    _b_PMISVD_vec_norm = _PMISVD_vecs[a]/np.linalg.norm(_PMISVD_vecs[b])
    _b_PPMISVD_vec_norm = _PPMISVD_vecs[a]/np.linalg.norm(_PPMISVD_vecs[b])
    _b_SPPMISVD_vec_norm = _SPPMISVD_vecs[a]/np.linalg.norm(_SPPMISVD_vecs[b])
    
    _center_PMISVD_vec = (_a_PMISVD_vec_norm + _b_PMISVD_vec_norm)/2
    _center_PPMISVD_vec = (_a_PPMISVD_vec_norm + _b_PPMISVD_vec_norm)/2
    _center_SPPMISVD_vec = (_a_SPPMISVD_vec_norm + _b_SPPMISVD_vec_norm)/2
    _matrix_PMISVD.append(_a_PMISVD_vec_norm - _center_PMISVD_vec)
    _matrix_PPMISVD.append(_a_PPMISVD_vec_norm - _center_PPMISVD_vec)
    _matrix_SPPMISVD.append(_a_SPPMISVD_vec_norm - _center_SPPMISVD_vec)
    _matrix_PMISVD.append(_b_PMISVD_vec_norm - _center_PMISVD_vec)
    _matrix_PPMISVD.append(_b_PPMISVD_vec_norm - _center_PPMISVD_vec)
    _matrix_SPPMISVD.append(_b_SPPMISVD_vec_norm - _center_SPPMISVD_vec)
    
_matrix_CoProb = np.array(_matrix_CoProb)
_matrix_PMI = np.array(_matrix_PMI)
_matrix_PPMI = np.array(_matrix_PPMI)
_matrix_SPPMI = np.array(_matrix_SPPMI)
_matrix_PMISVD = np.array(_matrix_PMISVD)
_matrix_PPMISVD = np.array(_matrix_PPMISVD)
_matrix_SPPMISVD = np.array(_matrix_SPPMISVD)

pca = PCA(n_components = 10)
gendir_CoProb_vec = pca.fit(_matrix_CoProb).components_[0]
gendir_PMI_vec = pca.fit(_matrix_PMI).components_[0]
gendir_PPMI_vec = pca.fit(_matrix_PPMI).components_[0]
gendir_SPPMI_vec = pca.fit(_matrix_SPPMI).components_[0]
gendir_PMISVD_vec = pca.fit(_matrix_PMISVD).components_[0]
gendir_PPMISVD_vec = pca.fit(_matrix_PPMISVD).components_[0]
gendir_SPPMISVD_vec = pca.fit(_matrix_SPPMISVD).components_[0]


/home/navid/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


## Measuring Gender Bias

In [64]:
job_CoProb_1st_mean_bias = []
job_PMI_1st_mean_bias = []
job_PPMI_1st_mean_bias = []
job_SPPMI_1st_mean_bias = []
for job_tuple in job_tuples:
    _vec = job_CoProb_vecs[job_tuple[0]]
    job_CoProb_1st_mean_bias.append((job_tuple[0], np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec) - np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec),
                                     np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec),
                                     np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec), job_tuple[1]))
    _vec = job_PMI_vecs[job_tuple[0]]
    job_PMI_1st_mean_bias.append((job_tuple[0], np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec) - np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec), 
                                  np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec),
                                  np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec), job_tuple[1]))
    _vec = job_PPMI_vecs[job_tuple[0]]
    job_PPMI_1st_mean_bias.append((job_tuple[0], np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec) - np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec),
                                   np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec),
                                   np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec), job_tuple[1]))
    _vec = job_SPPMI_vecs[job_tuple[0]]
    job_SPPMI_1st_mean_bias.append((job_tuple[0], np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec) - np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec),
                                    np.dot(_vec, coomtx_fem_exp_feature_vec)/np.sum(coomtx_fem_exp_feature_vec),
                                    np.dot(_vec, coomtx_masc_exp_feature_vec)/np.sum(coomtx_masc_exp_feature_vec), job_tuple[1]))


job_CoProb_1st_mean_bias.sort(key=lambda x: x[1])    
job_PMI_1st_mean_bias.sort(key=lambda x: x[1])    
job_PPMI_1st_mean_bias.sort(key=lambda x: x[1])    
job_SPPMI_1st_mean_bias.sort(key=lambda x: x[1])


In [65]:
                           
job_CoProb_2nd_gendir_bias = []
job_PMI_2nd_gendir_bias = []
job_PPMI_2nd_gendir_bias = []
job_SPPMI_2nd_gendir_bias = []
for job_tuple in job_tuples:
    _vec = job_CoProb_vecs[job_tuple[0]]
    job_CoProb_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_CoProb_vec), 0.0, 0.0, job_tuple[1]))
    
    _vec = job_PMI_vecs[job_tuple[0]]
    job_PMI_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_PMI_vec), 0.0, 0.0, job_tuple[1]))
    
    _vec = job_PPMI_vecs[job_tuple[0]]
    job_PPMI_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_PPMI_vec), 0.0, 0.0, job_tuple[1]))
    
    _vec = job_SPPMI_vecs[job_tuple[0]]
    job_SPPMI_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_SPPMI_vec), 0.0, 0.0, job_tuple[1]))
    

job_CoProb_2nd_gendir_bias.sort(key=lambda x: x[1])    
job_PMI_2nd_gendir_bias.sort(key=lambda x: x[1])    
job_PPMI_2nd_gendir_bias.sort(key=lambda x: x[1])    
job_SPPMI_2nd_gendir_bias.sort(key=lambda x: x[1])


In [50]:
                           
job_PMISVD_2nd_gendir_bias = []
job_PPMISVD_2nd_gendir_bias = []
job_SPPMISVD_2nd_gendir_bias = []
for job_tuple in job_tuples:
    _vec = job_PMISVD_vecs[job_tuple[0]]
    job_PMISVD_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_PMISVD_vec), 0.0, 0.0, job_tuple[1]))
    
    _vec = job_PPMISVD_vecs[job_tuple[0]]
    job_PPMISVD_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_PPMISVD_vec), 0.0, 0.0, job_tuple[1]))
    
    _vec = job_SPPMISVD_vecs[job_tuple[0]]
    job_SPPMISVD_2nd_gendir_bias.append((job_tuple[0], np.dot(_vec, gendir_SPPMISVD_vec), 0.0, 0.0, job_tuple[1]))
    

job_PMISVD_2nd_gendir_bias.sort(key=lambda x: x[1])    
job_PPMISVD_2nd_gendir_bias.sort(key=lambda x: x[1])    
job_SPPMISVD_2nd_gendir_bias.sort(key=lambda x: x[1])



In [67]:
job_CoProb_2nd_mean_bias = []
job_PMI_2nd_mean_bias = []
job_PPMI_2nd_mean_bias = []
job_SPPMI_2nd_mean_bias = []
for job_tuple in job_tuples:
    _vec = job_CoProb_vecs[job_tuple[0]]
    job_CoProb_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _CoProb_fem_vectors) - meanCosine(_vec, _CoProb_masc_vectors),
                                     meanCosine(_vec, _CoProb_masc_vectors), meanCosine(_vec, _CoProb_masc_vectors), job_tuple[1]))
    _vec = job_PMI_vecs[job_tuple[0]]
    job_PMI_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _PMI_fem_vectors) - meanCosine(_vec, _PMI_masc_vectors),
                                    meanCosine(_vec, _PMI_masc_vectors), meanCosine(_vec, _PMI_masc_vectors), job_tuple[1]))
    _vec = job_PPMI_vecs[job_tuple[0]]
    job_PPMI_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _PPMI_fem_vectors) - meanCosine(_vec, _PPMI_masc_vectors),
                                    meanCosine(_vec, _PPMI_fem_vectors), meanCosine(_vec, _PPMI_masc_vectors), job_tuple[1]))
    _vec = job_SPPMI_vecs[job_tuple[0]]
    job_SPPMI_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _SPPMI_fem_vectors) - meanCosine(_vec, _SPPMI_masc_vectors),
                                    meanCosine(_vec, _SPPMI_fem_vectors), meanCosine(_vec, _SPPMI_masc_vectors), job_tuple[1]))
    
job_CoProb_2nd_mean_bias.sort(key=lambda x: x[1])
job_PMI_2nd_mean_bias.sort(key=lambda x: x[1])
job_PPMI_2nd_mean_bias.sort(key=lambda x: x[1])
job_SPPMI_2nd_mean_bias.sort(key=lambda x: x[1])



In [52]:
job_PMISVD_2nd_mean_bias = []
job_PPMISVD_2nd_mean_bias = []
job_SPPMISVD_2nd_mean_bias = []
for job_tuple in job_tuples:
    _vec = job_PMISVD_vecs[job_tuple[0]]
    job_PMISVD_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _PMISVD_fem_vectors) - meanCosine(_vec, _PMISVD_masc_vectors),
                                    meanCosine(_vec, _PMISVD_fem_vectors), meanCosine(_vec, _PMISVD_masc_vectors), job_tuple[1]))
    _vec = job_PPMISVD_vecs[job_tuple[0]]
    job_PPMISVD_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _PPMISVD_fem_vectors) - meanCosine(_vec, _PPMISVD_masc_vectors),
                                    meanCosine(_vec, _PPMISVD_fem_vectors), meanCosine(_vec, _PPMISVD_masc_vectors), job_tuple[1]))
    _vec = job_SPPMISVD_vecs[job_tuple[0]]
    job_SPPMISVD_2nd_mean_bias.append((job_tuple[0], meanCosine(_vec, _SPPMISVD_fem_vectors) - meanCosine(_vec, _SPPMISVD_masc_vectors),
                                    meanCosine(_vec, _SPPMISVD_fem_vectors), meanCosine(_vec, _SPPMISVD_masc_vectors), job_tuple[1]))
    
job_PMISVD_2nd_mean_bias.sort(key=lambda x: x[1])
job_PPMISVD_2nd_mean_bias.sort(key=lambda x: x[1])
job_SPPMISVD_2nd_mean_bias.sort(key=lambda x: x[1])




In [74]:
masc_CoProb_centroid_vec

array([1.94418517e-02, 2.54952573e-06, 4.59751157e-08, ...,
       3.81460149e-08, 2.76831554e-08, 5.99047020e-08])

In [75]:
job_CoProb_2nd_centroid_bias = []
job_PMI_2nd_centroid_bias = []
job_PPMI_2nd_centroid_bias = []
job_SPPMI_2nd_centroid_bias = []
for job_tuple in job_tuples:
    _vec = job_CoProb_vecs[job_tuple[0]]
    job_CoProb_2nd_centroid_bias.append((job_tuple[0], Cosine(_vec, fem_CoProb_centroid_vec) - Cosine(_vec, masc_CoProb_centroid_vec),
                                     Cosine(_vec, fem_CoProb_centroid_vec), Cosine(_vec, masc_CoProb_centroid_vec), job_tuple[1]))
    _vec = job_PMI_vecs[job_tuple[0]]
    job_PMI_2nd_centroid_bias.append((job_tuple[0], Cosine(_vec, fem_PMI_centroid_vec) - Cosine(_vec, masc_PMI_centroid_vec),
                                    Cosine(_vec, fem_PMI_centroid_vec), Cosine(_vec, masc_PMI_centroid_vec), job_tuple[1]))
    _vec = job_PPMI_vecs[job_tuple[0]]
    job_PPMI_2nd_centroid_bias.append((job_tuple[0], Cosine(_vec, fem_PPMI_centroid_vec) - Cosine(_vec, masc_PPMI_centroid_vec),
                                    Cosine(_vec, fem_PPMI_centroid_vec), Cosine(_vec, masc_PPMI_centroid_vec), job_tuple[1]))
    _vec = job_SPPMI_vecs[job_tuple[0]]
    job_SPPMI_2nd_centroid_bias.append((job_tuple[0], Cosine(_vec, fem_SPPMI_centroid_vec) - Cosine(_vec, masc_SPPMI_centroid_vec),
                                    Cosine(_vec, fem_SPPMI_centroid_vec), Cosine(_vec, masc_SPPMI_centroid_vec), job_tuple[1]))
    
job_CoProb_2nd_centroid_bias.sort(key=lambda x: x[1])
job_PMI_2nd_centroid_bias.sort(key=lambda x: x[1])
job_PPMI_2nd_centroid_bias.sort(key=lambda x: x[1])
job_SPPMI_2nd_centroid_bias.sort(key=lambda x: x[1])




# Correlation to Statistics

In [68]:

winobias_stats=[]
with open('resources/occupations_stats_winobias.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        winobias_stats.append((_tuple[0], float(_tuple[1])))
print (len(winobias_stats))

census_stats=[]
with open('resources/occupations_stats_census.txt') as fr:
    for l in fr:
        _tuple=l.strip('\n').split(',')
        census_stats.append((_tuple[0], float(_tuple[1])))
print (len(census_stats))



40
96


In [76]:
from scipy.stats.stats import spearmanr
from scipy.stats.stats import pearsonr


def calc_correlations(_stats_jobs, _stats_bias):
    job_bias_data = {
        'job_PMISVD_2nd_gendir_bias':job_PMISVD_2nd_gendir_bias,
        'job_PPMISVD_2nd_gendir_bias':job_PPMISVD_2nd_gendir_bias,
        'job_SPPMISVD_2nd_gendir_bias':job_SPPMISVD_2nd_gendir_bias,
        'job_PMISVD_2nd_mean_bias':job_PMISVD_2nd_mean_bias,
        'job_PPMISVD_2nd_mean_bias':job_PPMISVD_2nd_mean_bias,
        'job_SPPMISVD_2nd_mean_bias':job_SPPMISVD_2nd_mean_bias,
        
        'job_PMI_2nd_gendir_bias':job_PMI_2nd_gendir_bias,
        'job_PMI_2nd_mean_bias':job_PMI_2nd_mean_bias,
        'job_PMI_2nd_centroid_bias':job_PMI_2nd_centroid_bias,
        'job_PMI_1st_mean_bias':job_PMI_1st_mean_bias,
        
        'job_PPMI_2nd_gendir_bias':job_PPMI_2nd_gendir_bias,
        'job_PPMI_2nd_mean_bias':job_PPMI_2nd_mean_bias,
        'job_PPMI_2nd_centroid_bias':job_PPMI_2nd_centroid_bias,
        'job_PPMI_1st_mean_bias':job_PPMI_1st_mean_bias,
        
        'job_SPPMI_2nd_gendir_bias':job_SPPMI_2nd_gendir_bias,
        'job_SPPMI_2nd_mean_bias':job_SPPMI_2nd_mean_bias,
        'job_SPPMI_2nd_centroid_bias':job_SPPMI_2nd_centroid_bias,
        'job_SPPMI_1st_mean_bias':job_SPPMI_1st_mean_bias,
                     
        'job_CoProb_2nd_gendir_bias':job_CoProb_2nd_gendir_bias,
        'job_CoProb_2nd_mean_bias':job_CoProb_2nd_mean_bias,
        'job_CoProb_2nd_centroid_bias':job_CoProb_2nd_centroid_bias,
        'job_CoProb_1st_mean_bias':job_CoProb_1st_mean_bias,
        
        }
    labor_bias = {}
    for _data_key in job_bias_data:
        labor_bias[_data_key] = []
    _idx = 1
    for _job in _stats_jobs:
        for _data_key in job_bias_data:
            labor_bias[_data_key].append([x for x in job_bias_data[_data_key] if x[0]==_job][0][_idx])
        
    results = {'spearman':{}, 'pearson':{}}
    print ('Spearman')
    for _data_key in job_bias_data:
        _text = _data_key.replace('job_', '').replace('_bias', '')
        results['spearman'][_text] = abs(round(spearmanr(labor_bias[_data_key], _stats_bias).correlation, 2))
        print ("%20s %0.2f" % (_text, results['spearman'][_text]))
    print ()
    print ('Pearson')
    for _data_key in job_bias_data:
        _text = _data_key.replace('job_', '').replace('_bias', '')
        results['pearson'][_text] = abs(round(pearsonr(labor_bias[_data_key], _stats_bias)[0], 2))
        print ("%20s %0.2f" % (_text, results['pearson'][_text]))
    print ()
    
    return results
    

results = {}

print ("WinoBias")
print ("----------")
results['winobias'] = calc_correlations([x[0] for x in winobias_stats], [x[1] for x in winobias_stats])
print ("**********")
#calc_correlations([x[0] for x in winobias_stats], [np.log((x[1]/100)/(1-(x[1]/100))) for x in winobias_stats])
#print ("**********")
#print ()
print ("U.S. Census Database")
#calc_correlations([x[0] for x in census_stats], [np.log(x[1]/(1-x[1])) for x in census_stats])
#print ("**********")
#print ()
results['census'] = calc_correlations([x[0] for x in census_stats], [x[1] for x in census_stats])
print ("**********")
print ()




WinoBias
----------
Spearman
   PMISVD_2nd_gendir 0.05
  PPMISVD_2nd_gendir 0.05
 SPPMISVD_2nd_gendir 0.17
     PMISVD_2nd_mean 0.41
    PPMISVD_2nd_mean 0.41
   SPPMISVD_2nd_mean 0.26
      PMI_2nd_gendir 0.28
        PMI_2nd_mean 0.33
    PMI_2nd_centroid 0.14
        PMI_1st_mean 0.53
     PPMI_2nd_gendir 0.45
       PPMI_2nd_mean 0.43
   PPMI_2nd_centroid 0.43
       PPMI_1st_mean 0.59
    SPPMI_2nd_gendir 0.26
      SPPMI_2nd_mean 0.32
  SPPMI_2nd_centroid 0.39
      SPPMI_1st_mean 0.57
   CoProb_2nd_gendir 0.22
     CoProb_2nd_mean 0.25
 CoProb_2nd_centroid 0.28
     CoProb_1st_mean 0.38

Pearson
   PMISVD_2nd_gendir 0.07
  PPMISVD_2nd_gendir 0.07
 SPPMISVD_2nd_gendir 0.29
     PMISVD_2nd_mean 0.49
    PPMISVD_2nd_mean 0.49
   SPPMISVD_2nd_mean 0.38
      PMI_2nd_gendir 0.07
        PMI_2nd_mean 0.24
    PMI_2nd_centroid 0.21
        PMI_1st_mean 0.51
     PPMI_2nd_gendir 0.49
       PPMI_2nd_mean 0.46
   PPMI_2nd_centroid 0.46
       PPMI_1st_mean 0.58
    SPPMI_2nd_gendir 0.37


In [77]:
for x in ['PMI_2nd_gendir','PMI_2nd_mean','PMI_2nd_centroid','PMI_1st_mean',
          'PPMI_2nd_gendir','PPMI_2nd_mean','PPMI_2nd_centroid','PPMI_1st_mean',
          'SPPMI_2nd_gendir','SPPMI_2nd_mean','SPPMI_2nd_centroid','SPPMI_1st_mean',
          'CoProb_2nd_gendir','CoProb_2nd_mean','CoProb_2nd_centroid','CoProb_1st_mean']:
    print ("%0.2f & %0.2f & %0.2f & %0.2f" % (results['winobias']['spearman'][x],
                                             results['winobias']['pearson'][x],
                                             results['census']['spearman'][x],
                                             results['census']['pearson'][x]))




0.28 & 0.07 & 0.18 & 0.02
0.33 & 0.24 & 0.27 & 0.19
0.14 & 0.21 & 0.35 & 0.40
0.53 & 0.51 & 0.57 & 0.62
0.45 & 0.49 & 0.39 & 0.47
0.43 & 0.46 & 0.45 & 0.52
0.43 & 0.46 & 0.45 & 0.50
0.59 & 0.58 & 0.64 & 0.64
0.26 & 0.37 & 0.26 & 0.28
0.32 & 0.40 & 0.44 & 0.48
0.39 & 0.45 & 0.45 & 0.48
0.57 & 0.49 & 0.52 & 0.48
0.22 & 0.23 & 0.29 & 0.31
0.25 & 0.26 & 0.30 & 0.32
0.28 & 0.28 & 0.31 & 0.33
0.38 & 0.42 & 0.46 & 0.53
